In [1]:
%pip install openai python-docx

Defaulting to user installation because normal site-packages is not writeable
  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached httpcore-1.0.9-py3-none-any.whl.metadata (21 kB)
  Using cached h11-0.16.0-py3-none-any.whl.metadata (8.3 kB)
   ---------------------------------------- 0.0/948.6 kB ? eta -:--:--
   - -------------------------------------- 30.7/948.6 kB 1.3 MB/s eta 0:00:01
   - ------------------------------------- 41.0/948.6 kB 960.0 kB/s eta 0:00:01
   -- ------------------------------------ 71.7/948.6 kB 491.5 kB/s eta 0:00:02
   ------ ------------------------------- 153.6/948.6 kB 833.5 kB/s eta 0:00:01
   ------- ------------------------------ 194.6/948.6 kB 841.6 kB/s eta 0:00:01
   --------- ---------------------------- 235.5/948.6 kB 901.1 kB/s eta 0:00:01
   ------------ ------------------------- 307.2/948.6 kB 999.9 kB/s eta 0:00:01
   ------------- ------------------------ 337.9/948.6 kB 952.6 kB/s eta 0:00:01
   ----------------- ---

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [2]:
import os
import openai
import docx

In [21]:
class Chatbot:
    """
    A class to encapsulate the RAG chatbot functionality,
    allowing for custom API endpoints.
    """
    def __init__(self, api_key, model="gpt-4.1-mini", base_url="https://api.metisai.ir/openai/v1"):
        """
        Initializes the Chatbot, setting up the OpenAI client with a custom base URL.
        """
        if not api_key:
            raise ValueError("OpenAI API key is required.")
        
        self.model = model
        try:
            self.client = openai.OpenAI(api_key=api_key, base_url=base_url)
        except Exception as e:
            print(f"Error initializing OpenAI client: {e}")
            self.client = None

    def load_knowledge_base(self, directory="knowledge_docs"):
        """
        Loads knowledge from all .docx files in a specified directory.
        It now chunks the document by blank lines (double newlines) to keep related paragraphs together.
        """
        all_chunks = []
        print(f"Loading knowledge from directory: '{directory}'...")
        if not os.path.exists(directory) or not os.path.isdir(directory):
            print(f"Error: The directory '{directory}' was not found.")
            print("Please create it and add your .docx knowledge files.")
            return []
        
        doc_files = [f for f in os.listdir(directory) if f.endswith(".docx")]
        if not doc_files:
            print(f"Warning: No .docx files found in the '{directory}' directory.")
            return []

        for filename in doc_files:
            filepath = os.path.join(directory, filename)
            try:
                doc = docx.Document(filepath)
                # Reconstruct the full text to handle chunks separated by blank lines
                full_text = "\n".join(p.text for p in doc.paragraphs)
                # Split by double newline (which signifies a blank line in the doc)
                chunks = [chunk.strip() for chunk in full_text.split('\n\n') if chunk.strip()]
                all_chunks.extend(chunks)
                print(f"  - Loaded {len(chunks)} chunks from {filename}")
            except Exception as e:
                print(f"  - Could not read {filename}. Skipping. Error: {e}")
        
        print(f"Total knowledge chunks loaded: {len(all_chunks)}")
        return all_chunks

    def print_knowledge_base(self, knowledge_chunks):
        """Prints the loaded knowledge base chunks for verification."""
        print("\n--- Knowledge Base Loaded ---")
        if not knowledge_chunks:
            print("The knowledge base is empty or could not be loaded.")
        else:
            for i, chunk in enumerate(knowledge_chunks):
                print(f"  Chunk {i+1}: {chunk[:80]}...")
        print("-----------------------------\n")

    def retrieve_relevant_chunks(self, query, knowledge_chunks):
        """
        Performs 'naive' retrieval by finding the chunk with the most keyword matches.
        """
        query_words = set(query.lower().split())
        best_chunk = ""
        max_score = 0

        if not knowledge_chunks:
            return "No knowledge base loaded."

        for chunk in knowledge_chunks:
            chunk_words = set(chunk.lower().split())
            score = len(query_words.intersection(chunk_words))
            if score > max_score:
                max_score = score
                best_chunk = chunk
        
        if max_score > 0:
            return best_chunk
        else:
            return knowledge_chunks[0]

    def generate_response(self, query, context):
        """
        Generates a response using the specified model and context.
        """
        if not self.client:
            return "OpenAI client is not initialized. Cannot generate response."

        system_prompt = """
        شما یک دستیار خدمات مشتری متخصص و دوستانه برای یک شرکت هستید. نام شما «پشتیبان» است.
        شما باید فقط و فقط بر اساس «زمینه» ارائه شده به «سوال» کاربر پاسخ دهید.
        اطلاعاتی را از خودتان اضافه نکنید. اگر زمینه برای پاسخ دادن کافی نیست،
        مودبانه بگویید که اطلاعات کافی برای پاسخ به آن سوال را ندارید.
        ***شما باید همیشه و فقط به زبان فارسی پاسخ دهید.***
        """
        
        user_prompt = f"زمینه: \"{context}\"\n\nسوال: \"{query}\""

        try:
            completion = self.client.chat.completions.create(
                model=self.model,
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": user_prompt}
                ],
                temperature=0.5,
            )
            return completion.choices[0].message.content
        except openai.APIError as e:
            print(f"OpenAI API Error: {e}")
            return "متاسفانه در ارتباط با هوش مصنوعی خطایی رخ داد. لطفا دوباره تلاش کنید."
        except Exception as e:
            print(f"An unexpected error occurred: {e}")
            return "یک خطای غیرمنتظره رخ داد. لطفا با پشتیبانی تماس بگیرید."

In [22]:
def run():
    """Main function to initialize and run the chatbot loop."""
    print("--- ربات چت فارسی با RAG و مدل سفارشی ---")

    api_key = "tpsg-HURAqpPOuLGZtEReVzj3unfTRXGp45o"
    if not api_key:
        print("Error: The OPENAI_API_KEY environment variable is not set.")
        print("Please set it before running the script.")
        return

    try:
        chatbot = Chatbot(api_key=api_key)
    except ValueError as e:
        print(e)
        return

    print("برای خروج 'خروج' را تایپ کنید")

    knowledge = chatbot.load_knowledge_base("C:/Users/USER/Desktop/chatbot-project/knowledge_docs")

    if not knowledge:
        return

    chatbot.print_knowledge_base(knowledge)

    while True:
        user_query = input("شما: ")
        if user_query.lower() == 'خروج':
            break

        context_chunk = chatbot.retrieve_relevant_chunks(user_query, knowledge)
        answer = chatbot.generate_response(user_query, context_chunk)

        print(f"جم: {answer}")

In [23]:
# print_knowledge_base(knowledge)

In [24]:
run()

--- ربات چت فارسی با RAG و مدل سفارشی ---
برای خروج 'خروج' را تایپ کنید
Loading knowledge from directory: 'C:/Users/USER/Desktop/knowledge_docs'...
  - Loaded 2 chunks from 01- درباره شرکت تابان انرژی- Rev02.docx
  - Loaded 15 chunks from 02- معرفی تابان در مجله - Rev05.docx
  - Loaded 1 chunks from 03- 07- فروشگاه.docx
  - Loaded 19 chunks from 05- سایــر.docx
Total knowledge chunks loaded: 37

--- Knowledge Base Loaded ---
  Chunk 1: شرکت تابان انرژی مازند سو
در راستای گسترش انرژی های تجدیدپذیر از جمله انرژی فتوو...
  Chunk 2: اهداف شرکت
استفاده از انرژی خورشیدی در راستای کاهش آلودگی محیط زیست.
تنوع بخشی د...
  Chunk 3: در راستای گسترش انرژی های تجدیدپذیر از جمله انرژی فتوولتائیک، بعنوان اولین تولید...
  Chunk 4: اهداف شرکت تابان انرژی مازند سو را می توان بطور خلاصه به شرح ذیل ارائه نمود.
است...
  Chunk 5: همانطور که در شکل نشان داده شده میزان تابش در بدترین نقطه ایران که متعلق به منطق...
  Chunk 6: ناترازی برق در زمان اوج مصرف از سال ۱۳۹۶ تا ۱۴۰۳
مهم‌ترین عوامل ناترازی برق در س...
 

شما:  اسم مدیریت شرکت چیست؟


جم: مدیر عامل شرکت تابان انرژی مازند سو دکتر احمدعلی غلامی است.


شما:  فروشگاه شامل چه محصولاتی است؟


جم: فروشگاه تابان انرژی مازند سو شامل محصولات زیر می‌باشد:
باتری خورشیدی، چراغ خورشیدی، اینورتر هیبریدی، اینورتر متصل به شبکه، اینورتر منفصل از شبکه، پنل خورشیدی و سایر لوازم و تجهیزات مرتبط.


شما:  برای کسب اطلاعات بیشتر چه کار کنیم؟


جم: برای کسب اطلاعات بیشتر می‌توانید با واحد فنی شرکت تماس حاصل نمایید. کارشناسان ما در هر زمان آماده ارائه خدمات طراحی، مشاوره و اجرای نیروگاه‌های خورشیدی در سراسر کشور هستند.


شما:  شاخص های زیست محیطی رو میگی؟


جم: با توجه به اطلاعات ارائه شده، شرکت تابان انرژی مازند سو با تمرکز بر استفاده از انرژی‌های تجدیدپذیر و پاک، بخصوص انرژی خورشیدی با تکنولوژی نانو وایر، به کاهش آلودگی‌های زیست‌محیطی ناشی از سوخت‌های فسیلی کمک می‌کند. این شرکت با تولید پنل‌های خورشیدی، به کاهش گرمایش زمین و تغییرات نگران‌کننده آب و هوایی کمک می‌نماید. بنابراین، شاخص‌های زیست محیطی مرتبط با فعالیت‌های این شرکت شامل کاهش انتشار گازهای گلخانه‌ای، کاهش آلودگی هوا، استفاده از منابع انرژی پاک و تجدیدپذیر و حمایت از توسعه پایدار می‌باشد. اگر نیاز به اطلاعات دقیق‌تر یا شاخص‌های خاص دارید، لطفاً اعلام بفرمایید.


شما:  خروج
